In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pprint as pp
import IPython
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
CSS = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(CSS))
pd.set_option('display.max_columns', None)

---

## Mapping categories to DF1 and removing outliers below

In [2]:
df1 = pd.read_csv('./data/0826export_column_filter_1.csv')
df1.columns = df1.columns.str.lower()

df2 = pd.read_excel('./documentation/store_master.xlsx')
df2.columns = df2.columns.str.lower()

categories_to_map = ['class', 
                     'store_city', 
                     'store_state', 
                     'close_date', 
                     'gross_feet'] # must be df2's LOWERED column names

def map_my_dataframe(df1, df2, a_list):
    """Returns DataFrame(df1)
    maps str items in a_list, from df2 onto df1.
    the joint variable for mapping is the store number.
    
    !! df1 and df2 column names must be lower-case !!
    """
    joint_var = 'store'
    for category in a_list:
        df_2_dict = dict(zip(df2[joint_var], df2[category]))
        df1[category] = df1[joint_var].map(df_2_dict)
    return df1
    
df = map_my_dataframe(df1, df2, categories_to_map)
df = df[df.store != 2084]

closed = df[df.close_date.notna()]

---

### Remove outlier store(2084), make DF for closed stores.

---

## Making dataframes to concat and compare below by `store`:

Grouping DataFrames's by `'store'`, `'mean()'` of:
 - `df1`: full original data frame
 - `top_avg`: top by `comp_pct`
 - `bot_avg`: bottom by `comp_pct`
 - `closed_avg`: by closed store

In [3]:
# all_avg = df.groupby(['store','fiscal_year','fiscal_month']).mean().median().round(3)
# top_avg = df.groupby(['store','fiscal_year','fiscal_month']).mean().nlargest(27, 'comp_pct').median().round(3)
# bot_avg = df.groupby(['store','fiscal_year','fiscal_month']).mean().nsmallest(27, 'comp_pct').median().round(3)
# closed_avg = closed.groupby(['store','fiscal_year','fiscal_month']).mean().median().round(3)

# # Concatenating
# for_compare = pd.concat([
#     all_avg.rename('all_means').to_frame(),
#     top_avg.rename('top_means').to_frame(),
#     bot_avg.rename('bot_means').to_frame(),
#     closed_avg.rename('closed_means').to_frame()
# ], axis=1).iloc[1:, :]

# for_compare

all_avg_by_store = df.groupby(['store']).mean().mean().round(4)
top_avg_by_store = df.groupby(['store']).mean().nlargest(20, 'comp_pct').mean().round(4)
bot_avg_by_store = df.groupby(['store']).mean().nsmallest(20, 'comp_pct').mean().round(4)
closed_avg_by_store = closed.groupby(['store']).mean().mean().round(4)

# Concatenating
for_compare_by_store = pd.concat([
#     all_avg_by_store.rename('all_means').to_frame(),
    top_avg_by_store.rename('top_store_avgs').to_frame(),
    bot_avg_by_store.rename('bot_store_avgs').to_frame(),
    closed_avg_by_store.rename('closed_store_avgs').to_frame()
], axis=1).iloc[3:, :]

for_compare_by_store

,top_store_avgs,bot_store_avgs,closed_store_avgs
comp_pct,0.1587,-0.0535,-0.0148
mark_down_amt_ty,2124.6918,1711.4393,1144.2139
payroll_adj_hours,143.3224,129.9134,106.2597
strak_traffic,1084.4834,905.5472,779.3127
strak_sales_amt,20370.3780,16344.4602,15008.3269
special_order_amt,3087.8884,2167.4445,2039.8378
sales_value,20368.6170,16340.8817,15003.0004
shoes_units,94.8406,80.0852,65.7381
shoes_value,12554.7852,10419.3385,9758.1475
multi_value,5018.2321,4499.7414,3764.7171


---
## Making dataframes to concat and compare below by `class`:

### Commented out classes do not have enough stores to do top and bot 10 store difference checks

In [4]:
classes = df['class'].unique()

mall_top = df[df['class'] == classes[0]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 3:].mean().rename('mall_top').to_frame()
mall_bot = df[df['class'] == classes[0]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 3:].mean().rename('mall_bot').to_frame()

# open_air_top = df[df['class'] == classes[1]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 3:].mean().rename('open_air_top').to_frame()
# open_air_bot = df[df['class'] == classes[1]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 3:].mean().rename('open_air_bot').to_frame()

# downtown_top = df[df['class'] == classes[2]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 3:].mean().rename('downtown_top').to_frame()
# downtown_bot = df[df['class'] == classes[2]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 3:].mean().rename('downtown_bot').to_frame()

# street_top = df[df['class'] == classes[3]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 3:].mean().rename('street_top').to_frame()
# street_bot = df[df['class'] == classes[3]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 3:].mean().rename('street_bot').to_frame()

airport_top = df[df['class'] == classes[4]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 3:].mean().rename('airport_top').to_frame()
airport_bot = df[df['class'] == classes[4]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 3:].mean().rename('airport_bot').to_frame()

# lifestyle_top = df1[df1['class'] == classes[5]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 3:].mean().rename('lifestyle_top').to_frame()
# lifestyle_bot = df1[df1['class'] == classes[5]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 3:].mean().rename('lifestyle_bot').to_frame()

outlet_top = df[df['class'] == classes[6]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 3:].mean().rename('outlet_top').to_frame()
outlet_bot = df[df['class'] == classes[6]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 3:].mean().rename('outlet_bot').to_frame()

for_compare_by_class = pd.concat([mall_top, mall_bot, airport_top, airport_bot, outlet_top, outlet_bot], axis=1)
for_compare_by_class

,mall_top,mall_bot,airport_top,airport_bot,outlet_top,outlet_bot
comp_pct,0.148012,-0.057884,0.120980,0.035684,0.125864,-0.026232
mark_down_amt_ty,1571.621937,1216.257061,3403.726138,2430.833884,2961.037824,2886.487270
payroll_adj_hours,137.263298,139.608073,188.585432,156.397137,127.517861,111.503552
strak_traffic,1038.946692,964.761647,2436.505746,1783.348840,970.653400,858.412714
strak_sales_amt,20377.019085,16116.253101,33671.195384,24858.006771,16006.193421,13609.018810
special_order_amt,3498.733135,2452.096504,4132.429665,3383.448032,1707.549570,1540.681847
sales_value,20372.284516,16112.981521,33671.396492,24864.942719,16004.955038,13604.867630
shoes_units,88.875917,68.489534,101.814827,74.808826,114.120242,94.176681
shoes_value,13346.021536,10149.231473,14299.380063,10748.085744,11001.005083,8992.825135
multi_value,5302.006192,3766.266602,4388.615398,3162.557390,5303.936285,4725.117806


---

## MALL STORES:
 - #### Absolute percentage(%) differences between averages of top and bot mall stores' variables:

In [5]:
top = mall_top.iloc[1:,0]
bot = mall_bot.iloc[1:,0]
diff_mall = abs(abs(bot - top) / ((bot + top) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]

---

## AIRPORT STORES:
 - #### Absolute percentage(%) differences between averages of top and bot airport stores' variables:

In [6]:
top = airport_top.iloc[1:,0]
bot = airport_bot.iloc[1:,0]
diff_airport = abs(abs(bot-top) / ((bot + top) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]

---

## OUTLET STORES: 
 - #### Absolute percentage(%) differences between averages of top and bot outlet stores' variables:

In [7]:
top = outlet_top.iloc[1:,0]
bot = outlet_bot.iloc[1:,0]

# outlet store differences
diff_outlet = abs(abs(top-bot) / ((top + bot) * 0.5) * 100).to_frame('abs_%_diff').nlargest(10, 'abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]

---

## Making dataframes to concat and compare below by `fiscal_week`, top and bot `comp_pct`.

Each observation within `df1` is `fiscal_week`, and total len is 27042. We will sample top and bot 1000 fiscal weeks by `comp_pct`, then compare the variable of these fiscal weeks with side by side.

In [8]:
fiscal_wk_top = df.set_index(['store', 'fiscal_week']).nlargest(1000, 'comp_pct').iloc[:,2:]
fiscal_wk_bot = df.set_index(['store', 'fiscal_week']).nsmallest(1000, 'comp_pct').iloc[:,2:]


for_compare_by_fiscal_week = pd.concat([fiscal_wk_top.mean().rename('top_comp_pct_weeks'), 
                                        fiscal_wk_bot.mean().rename('bot_comp_pct_weeks')], axis=1)
for_compare_by_fiscal_week

,top_comp_pct_weeks,bot_comp_pct_weeks
comp_pct,0.956945,-0.436951
mark_down_amt_ty,2315.725860,1167.420960
payroll_adj_hours,130.121000,118.318000
strak_traffic,950.349000,704.071000
strak_sales_amt,19588.119570,10442.047940
special_order_amt,2834.251660,1497.460080
sales_value,19585.374860,10423.961340
shoes_units,102.545000,56.047000
shoes_value,12795.650900,6951.279400
multi_value,5961.896140,2716.021310


---

## FISCAL WEEKS: 
 - #### Absolute percentage(%) differences between averages of top and bot fiscal weeks' variables:

In [9]:
wk_top = fiscal_wk_top.mean()[1:]
wk_bot = fiscal_wk_bot.mean()[1:]

diff_week = abs(abs(wk_top - wk_bot) / ((wk_top + wk_bot) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]